# Covid-19: Hospitals in New York

Numerous reports on severe shortages of medical supplies in USA from Covid-19 outbreak. This study aims to optimizing medical supplies for the hospitals in New York. Medical supplies can be masks, PPE suits or ventilators.

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 79 kB 7.1 MB/s eta 0:00:011
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=92c13af7eca5b08a5e45356b22f84cfd784d37bbf7b5d062c7bcc1149224e0e5
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Folium installed
Libraries imported.


# Data Acquisition

This study will use Foursquare to retrieve the data and assume one of the hospital in the stated address below has sufficient medical supplies to spare.

In [5]:
CLIENT_ID = 'YZ4NXRS4PYCPAPDGIO4YRQRRVKGOYD1GUGPUVUL5Z15KDMJU' # your Foursquare ID
CLIENT_SECRET = 'M2OQQ4QVYGB0FX2M2YKYDL411DQJ5VBDKPYHXB31LFZF0MO4' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YZ4NXRS4PYCPAPDGIO4YRQRRVKGOYD1GUGPUVUL5Z15KDMJU
CLIENT_SECRET:M2OQQ4QVYGB0FX2M2YKYDL411DQJ5VBDKPYHXB31LFZF0MO4


In [6]:
address = '180 Fort Washington Ave, New York, NY 10032, United States'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.8412456 -73.9424039


In [7]:
search_query = 'Hospital'
radius = 500
print(search_query + ' .... OK!')

Hospital .... OK!


In [8]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YZ4NXRS4PYCPAPDGIO4YRQRRVKGOYD1GUGPUVUL5Z15KDMJU&client_secret=M2OQQ4QVYGB0FX2M2YKYDL411DQJ5VBDKPYHXB31LFZF0MO4&ll=40.8412456,-73.9424039&v=20180604&query=Hospital&radius=500&limit=100'

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f8ad87e418d9541e723e97e'},
 'response': {'venues': [{'id': '4bbe05aca8cf76b0b07fb2fd',
    'name': 'Milstein Hospital Building',
    'location': {'address': '177 Fort Washington Ave',
     'lat': 40.8416387030687,
     'lng': -73.94340634346008,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.8416387030687,
       'lng': -73.94340634346008}],
     'distance': 95,
     'postalCode': '10032',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['177 Fort Washington Ave',
      'New York, NY 10032',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d196941735',
      'name': 'Hospital',
      'pluralName': 'Hospitals',
      'shortName': 'Hospital',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1602934910',
    'hasPerk': False},
   {'id': '4ac8b

# Data Processing and Cleaning

Putting into pandas data frame and keep the relevant information and columns.

In [10]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4bbe05aca8cf76b0b07fb2fd,Milstein Hospital Building,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1602934910,False,177 Fort Washington Ave,40.841639,-73.943406,"[{'label': 'display', 'lat': 40.8416387030687,...",95,10032,US,New York,NY,United States,"[177 Fort Washington Ave, New York, NY 10032, ...",NaN
1,4ac8b592f964a52054bc20e3,New York-Presbyterian Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1602934910,False,622 W 168th St,40.841135,-73.941743,"[{'label': 'display', 'lat': 40.84113547326098...",56,10032,US,New York,NY,United States,"[622 W 168th St (at Fort Washington Ave), New ...",at Fort Washington Ave
2,4b5f5f5af964a520abb629e3,Morgan Stanley Children's Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1602934910,False,3959 Broadway,40.839674,-73.941411,"[{'label': 'display', 'lat': 40.83967446184869...",193,10032,US,New York,NY,United States,"[3959 Broadway (at W 165th St), New York, NY 1...",at W 165th St
3,50c499cde4b0c7bb40d04ef6,CT - ICU @ NewYork-Presbyterian Hospital,"[{'id': '58daa1558bbb0b01f18ec1f7', 'name': 'H...",v-1602934910,False,NaN,40.841552,-73.943981,"[{'label': 'display', 'lat': 40.84155209807606...",137,10032,US,New York,NY,United States,"[New York, NY 10032, United States]",NaN
4,4bf2a23a77b4c92894786a1c,New York Orthopaedic Hospital Associates,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",v-1602934910,False,622 W 168th St,40.840464,-73.941673,"[{'label': 'display', 'lat': 40.84046371046147...",106,10032,US,New York,NY,United States,"[622 W 168th St, New York, NY 10032, United St...",NaN


In [11]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Milstein Hospital Building,Hospital,177 Fort Washington Ave,40.841639,-73.943406,"[{'label': 'display', 'lat': 40.8416387030687,...",95,10032,US,New York,NY,United States,"[177 Fort Washington Ave, New York, NY 10032, ...",NaN,4bbe05aca8cf76b0b07fb2fd
1,New York-Presbyterian Hospital,Hospital,622 W 168th St,40.841135,-73.941743,"[{'label': 'display', 'lat': 40.84113547326098...",56,10032,US,New York,NY,United States,"[622 W 168th St (at Fort Washington Ave), New ...",at Fort Washington Ave,4ac8b592f964a52054bc20e3
2,Morgan Stanley Children's Hospital,Hospital,3959 Broadway,40.839674,-73.941411,"[{'label': 'display', 'lat': 40.83967446184869...",193,10032,US,New York,NY,United States,"[3959 Broadway (at W 165th St), New York, NY 1...",at W 165th St,4b5f5f5af964a520abb629e3
3,CT - ICU @ NewYork-Presbyterian Hospital,Hospital Ward,NaN,40.841552,-73.943981,"[{'label': 'display', 'lat': 40.84155209807606...",137,10032,US,New York,NY,United States,"[New York, NY 10032, United States]",NaN,50c499cde4b0c7bb40d04ef6
4,New York Orthopaedic Hospital Associates,Medical Center,622 W 168th St,40.840464,-73.941673,"[{'label': 'display', 'lat': 40.84046371046147...",106,10032,US,New York,NY,United States,"[622 W 168th St, New York, NY 10032, United St...",NaN,4bf2a23a77b4c92894786a1c
5,NewYork-Presbyterian Morgan Stanley Children's...,Hospital,3959 Broadway,40.839845,-73.941379,"[{'label': 'display', 'lat': 40.83984471317083...",178,10032,US,New York,NY,United States,"[3959 Broadway (165th Street), New York, NY 10...",165th Street,518c3360498e55541832c6f9
6,Joint Center at University Hospital and Medica...,Medical Center,7201 N University Dr,40.844682,-73.940931,"[{'label': 'display', 'lat': 40.844682, 'lng':...",402,33321,US,Tamarac,FL,United States,"[7201 N University Dr, Tamarac, FL 33321, Unit...",NaN,5655eece498e3859a4b4af3d
7,The Garden at New York Presbyterian Hospital,Garden,NaN,40.840328,-73.941994,"[{'label': 'display', 'lat': 40.84032821655273...",107,NaN,US,New York,NY,United States,"[New York, NY, United States]",NaN,52335b9511d222d563eecaa7
8,Columbia Hospital Morgue,Medical Center,NaN,40.841160,-73.942000,"[{'label': 'display', 'lat': 40.84116, 'lng': ...",35,NaN,US,New York,NY,United States,"[New York, NY, United States]",NaN,58c72ec56ad5a12a6002bdf5
9,Harkness Pavilion,Hospital,180 Fort Washington Ave,40.841306,-73.942237,"[{'label': 'display', 'lat': 40.84130592249472...",15,10032,US,New York,NY,United States,"[180 Fort Washington Ave (at W 168th St), New ...",at W 168th St,4c1a22236a21c9b6528bcb97


In [12]:
dataframe_filtered.name

0                            Milstein Hospital Building
1                        New York-Presbyterian Hospital
2                    Morgan Stanley Children's Hospital
3              CT - ICU @ NewYork-Presbyterian Hospital
4              New York Orthopaedic Hospital Associates
5     NewYork-Presbyterian Morgan Stanley Children's...
6     Joint Center at University Hospital and Medica...
7          The Garden at New York Presbyterian Hospital
8                              Columbia Hospital Morgue
9                                     Harkness Pavilion
10                    NY Presbyterian Hospital Columbia
11                                  NYC EMS Hospital 17
12              New York-Presbyterian/Columbia Hospital
13                      Presbyterian Hospital Cafeteria
14                          Children's hospital 5 Tower
15    Bright Horizons at New York Presbyterian Hospi...
16               NewYork-Presbyterian Hospital Mycology
17                       Columbia Presbyterian H

# Results shown in Map

In [15]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around New York Hospital

# add a red circle marker to represent the Hospital
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='New York Hospital',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# Discussion & Conclusion

From the map, there are overlapping blue circles which may be under the same hospital management as we study the names in detail. Hospitals specializing in Orthopedic or Psychiatric are under this same hospital list is less relevant for this study and could be omitted.
To take this study to a higher level we can take further steps by mapping population to check population density of the area and priority can be given to hospitals with higher population around the area.

This preliminary study will allow optimization of constrained resources and effective allocation can be done to ensure more citizens has better chances of getting their medical supplies which will reduce the spread of virus and save their lives.
